In [1]:
"""
Created on Wed Feb 26 17:01:56 2020

"""

""" IMPORTS """
import sys
sys.path.insert(1,'../')
import numpy as np
np.random.seed(1337)  # for reproducibility
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.datasets import mnist
from tensorflow.keras.constraints import *
from sklearn.model_selection import train_test_split
# from keras.utils import np_utils

from binary_ops import binary_tanh as binary_tanh_op
from binary_layers import BinaryDense, BinaryConv2D

import h5py
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
from lambda_layers import *
from binary_ops import *
%matplotlib qt

In [2]:
""" FUNCTIION AND VARIABLE DEFINITIONS """
def binary_tanh(x):
    return binary_tanh_op(x)

H = 1.
kernel_lr_multiplier = 'Glorot'

# # nn
batch_size = 50
epochs = 20
channels = 1
img_rows = 30
img_cols = 30
filters = 32
kernel_size = (32, 32)
pool_size = (2, 2)
hidden_units = 128
classes = 10
use_bias = False

# # learning rate schedule
lr_start = 1e-3
lr_end = 1e-4
lr_decay = (lr_end / lr_start)**(1. / epochs)

# # BN
epsilon = 1e-6
momentum = 0.9

# # dropout
p1 = 0.25
p2 = 0.5

hdf5_dir = Path("../../data/hdf5/")

def read_many_hdf5(num_images):
    """ Reads image from HDF5.
        Parameters:
        ---------------
        num_images   number of images to read
        Returns:
        ----------
        images      images array, (N, 32, 32, 3) to be stored
        labels      associated meta data, int label (N, 1)
    """
    images= []

    # Open the HDF5 file
    file = h5py.File(hdf5_dir / f"{num_images}_vids.h5", "r+")

    images = np.array(file["/images"]).astype("float32")

    return images

def np_streak(x):
    input_dims = np.shape(x)
    output_shape = (input_dims[0],input_dims[1],input_dims[1]+input_dims[2],input_dims[3],input_dims[4])
    streak_tensor = np.zeros(output_shape)
    for i in range(output_shape[0]):
        for j in range(output_shape[1]):
            streak_tensor[i,j,j:(output_shape[3]+j),:,:] = x[i,j,:,:,:]
    #return streak_tensor
    return np.sum(streak_tensor,axis=1)

def mask(val,ims,mask):
    for i in range(np.shape(val)[0]):
        for j in range(np.shape(val)[1]):
            val[i,j,:,:] = ims[i,j,:,:] * mask
    return val



ims = read_many_hdf5('training')


ims = ims[:1500]
ims = np.reshape(ims,(-1,30,32,32,1))

validate = ims

validate = validate / 255
#ims2 = ims2 /255
ims = ims/255
#X_train, X_test, y_train, y_test = train_test_split(ims, validate, test_size=(1/3), random_state=42)
#X_train, X_test, y_train, y_test = train_test_split(ims2, validate, test_size=(1/3), random_state=42)

MX_train, MX_test, My_train, My_test = train_test_split(ims,ims, test_size = 0.3, random_state = 42)

print(np.shape(MX_test))
print(np.shape(MX_train))


reduce_lr = ReduceLROnPlateau(monitor='val_loss',verbose=1, factor=0.5,
                              patience=50, min_lr=0.000001)
early_stopping = EarlyStopping(patience=90,verbose=1,restore_best_weights=True)   


def custom_loss(y_true, y_pred):

  ssim_loss = (1.0-tf.image.ssim(y_true,y_pred,1))/2.0
  mse_loss = K.mean(K.square(y_pred-y_true))
  #mse_loss = tf.keras.losses.mean_squared_error(y_true,y_pred)

  ssim_loss = 0.5*ssim_loss
  mse_loss = 0.5*mse_loss

  return ssim_loss + mse_loss

def ssim_loss(y_true,y_pred):  
    return (1.0-tf.image.ssim(y_true,y_pred,1))/2.0

def mse_loss(y_true,y_pred):
    return K.mean(K.square(y_pred-y_true))


(450, 30, 32, 32, 1)
(1050, 30, 32, 32, 1)


In [3]:
""" VIDEO FUNCTIONS FOR CHECKING POST TRAINING"""
import os
try:
    os.mkdir('testing_ims')
except:
    pass

def get_mask(model,l=0):
    b = binarize(model.layers[l].weights[0])
    figb,axb = plt.subplots(1,1)
    axb.imshow(np.reshape(b,(32,32)),cmap="gray")
                            
def show_video(y_pred,y_true,num,filename,save=False):
    yp = np.reshape(y_pred,(-1,30,32,32))
    yt = np.reshape(y_true,(-1,30,32,32))
    split = np.zeros((5,32,32))
    yp_tensor = tf.convert_to_tensor(y_pred)
    yt_tensor = tf.convert_to_tensor(y_true)
    ssim = np.mean(ssim_loss(yt_tensor[num],yp_tensor[num]))
    mse = np.mean(mse_loss(yt_tensor[num],yp_tensor[num]))
    fig,ax = plt.subplots(nrows=5,ncols=13,figsize=(21,10),sharex=True,sharey=True)
    fig.suptitle(f'Movie: {num} MSE: {mse:.3} SSIM: {ssim:.3}')
    for row in range(5):
        for col in range(13):
            if col < 6:
                ax[row,col].imshow(yp[num][5*row+col],cmap="gray")
            elif col == 6:
                pass
            else:
                ax[row,col].imshow(yt[num][5*row + (col %6)],cmap="gray")
    if save:
        fig.savefig('testing_ims/'+filename+'.png')

def show_all_videos(videos,rows,cols):
    yp = np.reshape(videos, (-1,30,32,32))
    fix3,ax3 = plt.subplots(nrows=rows, ncols = cols)
    for row in range(rows):
        for col in range(cols):
            ax3[row,col].imshow(yp[rows*row+col][3],cmap="gray")

In [4]:
""" FORWARD MODEL """

forward_model = Sequential()
forward_model.add(Input(shape=(30,32,32,1),batch_size = 32))
forward_model.add(TimeDistributed(BinaryConv2D(1, kernel_size=(32,32), input_shape=(30,32,32,1),
                       data_format='channels_last',
                       H=H, kernel_lr_multiplier=kernel_lr_multiplier,
                       padding='same', use_bias=use_bias, name='bin_conv_1')))
forward_model.add(Reshape((30,32,32)))
forward_model.add(Lambda(streak,output_shape=streak_output_shape))
forward_model.add(Lambda(integrate_ims, output_shape = integrate_ims_output_shape))
forward_model.add(Flatten())
forward_model.add(Dense(30720, activation = 'relu'))
forward_model.add(Reshape((30,32,32,1)))
forward_model.compile(optimizer = Nadam(0.0001), loss = custom_loss, metrics = ['mean_squared_error',mse_loss])
forward_model.summary()


forward_model.load_weights('../../data/model_stuff/forward_weights_4_12.h5')
binary_weights = forward_model.layers[0].get_weights()
inverse_weights = forward_model.layers[5].get_weights()

(32, 30, 62, 32)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (32, 30, 32, 32, 1)       1024      
_________________________________________________________________
reshape (Reshape)            (32, 30, 32, 32)          0         
_________________________________________________________________
lambda (Lambda)              (32, 30, 62, 32)          0         
_________________________________________________________________
lambda_1 (Lambda)            (32, 62, 32)              0         
_________________________________________________________________
flatten (Flatten)            (32, 1984)                0         
_________________________________________________________________
dense (Dense)                (32, 30720)               60979200  
_________________________________________________________________
reshape_1 (Reshape)          (32, 30, 3

In [5]:
""" 
UNET MODEL
Fixing the weights for the bin_conv1 layer as well as the dense1 layer, ie NON TRAINABLE
Feeding in weights from the forward_model above to see if that improves the results from previous session

"""
inputs = Input(shape=(30,32,32,1))
bin_conv1 = TimeDistributed(BinaryConv2D(1, kernel_size=(32,32), input_shape=(30,32,32,1),
                       data_format='channels_last',
                       H=H, kernel_lr_multiplier=kernel_lr_multiplier,
                       padding='same', use_bias=use_bias, name='bin_conv_1',trainable=False))(inputs)
resh1 = Reshape((30,32,32))(bin_conv1)
s = Lambda(streak, output_shape = streak_output_shape)(resh1)
i = Lambda(integrate_ims, output_shape = integrate_ims_output_shape) (s)
f = Flatten()(i)
dense1 = Dense(30720, activation = 'relu',trainable=False)(f)
resh2 = Reshape((30,32,32,1))(dense1)
c1 = TimeDistributed(Conv2D(1, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (resh2)
c1 = Dropout(0.1) (c1)
c1 = TimeDistributed(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(c1)
p1 = TimeDistributed(MaxPooling2D((2, 2)))(c1)

c2 = TimeDistributed(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))(p1)
c2 = Dropout(0.1) (c2)
c2 = TimeDistributed(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(c2)
p2 = TimeDistributed(MaxPooling2D((2, 2)) )(c2)

c3 = TimeDistributed(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(p2)
c3 = Dropout(0.2) (c3)
c3 = TimeDistributed(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(c3)
p3 = TimeDistributed(MaxPooling2D((2, 2)) )(c3)
    
c4 = TimeDistributed(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(p3)
c4 = Dropout(0.2) (c4)
c4 = TimeDistributed(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(c4)
p4 = TimeDistributed(MaxPooling2D(pool_size=(2, 2))) (c4)

c5 = TimeDistributed(Conv2D(256, (2, 2), activation='relu', kernel_initializer='he_normal', padding='same')) (p4)
c5 = Dropout(0.3) (c5)
c5 = TimeDistributed(Conv2D(256, (2, 2), activation='relu', kernel_initializer='he_normal', padding='same')) (c5)

u6 = TimeDistributed(Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same'))(c5)
u6 = concatenate([u6, c4])
c6 = TimeDistributed( Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(u6)
c6 = Dropout(0.2) (c6)
c6 = TimeDistributed(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (c6)

u7 = TimeDistributed(Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same') )(c6)
u7 = concatenate([u7, c3])
c7 = TimeDistributed(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (u7)
c7 = Dropout(0.2) (c7)
c7 = TimeDistributed(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (c7)
    
u8 = TimeDistributed(Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same') )(c7)
u8 = concatenate([u8, c2])
c8 = TimeDistributed(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (u8)
c8 = Dropout(0.1) (c8)
c8 = TimeDistributed(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (c8)

u9 = TimeDistributed(Conv2DTranspose(16, (3, 3), strides=(2, 2), padding='same')) (c8)
c9 = TimeDistributed(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (u9)
c9 = Dropout(0.1) (c9)
c9 = TimeDistributed(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (c9)
    
outputs = TimeDistributed(Conv2D(1, (1, 1), activation='sigmoid')) (c9)

CUPNET = Model(inputs = [inputs], outputs = [outputs])
    
CUPNET.compile(optimizer = Nadam(), loss = custom_loss, metrics = ['mean_squared_error'])

(None, 30, 62, 32)


In [6]:
import os
print(os.getcwd())
models = os.listdir('../../data/model_stuff/cupnet_weights')
final= []
for model in models: 
    t = model.split('_')
    if t[3] == "lrs" and t[1] in ['0.0001','0.001']:
        final.append(model)
    elif float(t[1]) <= 0.7 and float(t[1]) >= 0.3:
        final.append(model)
print(final)

C:\Users\f002q97\Documents\GitHub\CUP-Net\src\jupyter_files
['0_0.0001_cupnet_lrs_4_12.h5', '0_0.001_cupnet_lrs_4_12.h5', '0_0.3_cupnet_4_12.h5', '0_0.3_cupnet_4_13.h5', '0_0.4_cupnet_4_12.h5', '0_0.4_cupnet_4_13.h5', '0_0.5_cupnet_4_12.h5', '0_0.5_cupnet_4_13.h5', '0_0.6_cupnet_4_12.h5', '0_0.6_cupnet_4_13.h5', '0_0.7_cupnet_4_12.h5', '0_0.7_cupnet_4_13.h5', '1_0.0001_cupnet_lrs_4_12.h5', '1_0.001_cupnet_lrs_4_12.h5', '1_0.3_cupnet_4_12.h5', '1_0.3_cupnet_4_13.h5', '1_0.4_cupnet_4_12.h5', '1_0.4_cupnet_4_13.h5', '1_0.5_cupnet_4_12.h5', '1_0.5_cupnet_4_13.h5', '1_0.6_cupnet_4_12.h5', '1_0.6_cupnet_4_13.h5', '1_0.7_cupnet_4_12.h5', '1_0.7_cupnet_4_13.h5']


In [8]:
CUPNET.load_weights('../../data/model_stuff/cupnet_weights/0_0.5_cupnet_4_12.h5')
CUPNET.layers[1].set_weights(binary_weights)
CUPNET.layers[6].set_weights(inverse_weights)
CUPNET.summary()
    
y_pred = CUPNET.predict(MX_test)
show_video(y_pred,My_test,15,model,True)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 30, 32, 32,  0                                            
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, 30, 32, 32, 1 1024        input_2[0][0]                    
__________________________________________________________________________________________________
reshape_2 (Reshape)             (None, 30, 32, 32)   0           time_distributed_1[0][0]         
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 30, 62, 32)   0           reshape_2[0][0]                  
______________________________________________________________________________________________

InternalError:  Blas xGEMMBatched launch failed : a.shape=[30,62,62], b.shape=[960,62,32], m=62, n=32, k=62, batch_size=960
	 [[{{node model/lambda_2/PartitionedCall/MatMul}}]] [Op:__inference_distributed_function_2551]

Function call stack:
distributed_function


In [7]:

for model in final:
    CUPNET.load_weights('../../data/model_stuff/cupnet_weights/'+model)
    CUPNET.layers[1].set_weights(binary_weights)
    CUPNET.layers[6].set_weights(inverse_weights)
    CUPNET.summary()
    
    y_pred = CUPNET.predict(MX_test)
    show_video(y_pred,My_test,30,model,True)



In [8]:
""" SET THE PREFIXED WEIGHTS FROM THE FORWARD MODEL """
CUPNET.layers[1].set_weights(binary_weights)
CUPNET.layers[6].set_weights(inverse_weights)
CUPNET.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 30, 32, 32,  0                                            
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, 30, 32, 32, 1 1024        input_2[0][0]                    
__________________________________________________________________________________________________
reshape_2 (Reshape)             (None, 30, 32, 32)   0           time_distributed_1[0][0]         
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 30, 62, 32)   0           reshape_2[0][0]                  
______________________________________________________________________________________________

In [ ]:
""" CUPNET TRAINING """
CUPNET_history = CUPNET.fit(MX_train, My_train,
          batch_size = 32,epochs= 200,
          verbose=2,validation_data=(MX_test,My_test),callbacks=[reduce_lr])

Train on 1050 samples, validate on 450 samples
Epoch 1/200
1050/1050 - 15s - loss: 0.1518 - mean_squared_error: 0.0204 - val_loss: 0.1786 - val_mean_squared_error: 0.0240
Epoch 2/200
1050/1050 - 15s - loss: 0.1452 - mean_squared_error: 0.0178 - val_loss: 0.1705 - val_mean_squared_error: 0.0199
Epoch 3/200
1050/1050 - 15s - loss: 0.1405 - mean_squared_error: 0.0164 - val_loss: 0.1677 - val_mean_squared_error: 0.0182
Epoch 4/200
1050/1050 - 15s - loss: 0.1378 - mean_squared_error: 0.0158 - val_loss: 0.1713 - val_mean_squared_error: 0.0209
Epoch 5/200
1050/1050 - 15s - loss: 0.1343 - mean_squared_error: 0.0151 - val_loss: 0.1688 - val_mean_squared_error: 0.0215
Epoch 6/200
1050/1050 - 15s - loss: 0.1304 - mean_squared_error: 0.0144 - val_loss: 0.1645 - val_mean_squared_error: 0.0178
Epoch 7/200
1050/1050 - 15s - loss: 0.1286 - mean_squared_error: 0.0141 - val_loss: 0.1636 - val_mean_squared_error: 0.0177
Epoch 8/200
1050/1050 - 15s - loss: 0.1266 - mean_squared_error: 0.0135 - val_loss: 0

Epoch 67/200
1050/1050 - 15s - loss: 0.0972 - mean_squared_error: 0.0094 - val_loss: 0.1542 - val_mean_squared_error: 0.0145
Epoch 68/200
1050/1050 - 15s - loss: 0.0959 - mean_squared_error: 0.0090 - val_loss: 0.1572 - val_mean_squared_error: 0.0167
Epoch 69/200
1050/1050 - 15s - loss: 0.0968 - mean_squared_error: 0.0095 - val_loss: 0.1554 - val_mean_squared_error: 0.0153
Epoch 70/200
1050/1050 - 15s - loss: 0.0954 - mean_squared_error: 0.0089 - val_loss: 0.1570 - val_mean_squared_error: 0.0165
Epoch 71/200
1050/1050 - 15s - loss: 0.0963 - mean_squared_error: 0.0094 - val_loss: 0.1544 - val_mean_squared_error: 0.0149
Epoch 72/200
1050/1050 - 15s - loss: 0.0953 - mean_squared_error: 0.0090 - val_loss: 0.1545 - val_mean_squared_error: 0.0146
Epoch 73/200
1050/1050 - 15s - loss: 0.0955 - mean_squared_error: 0.0094 - val_loss: 0.1563 - val_mean_squared_error: 0.0159
Epoch 74/200
1050/1050 - 15s - loss: 0.0946 - mean_squared_error: 0.0089 - val_loss: 0.1557 - val_mean_squared_error: 0.0152


In [ ]:
y_pred = CUPNET.predict(MX_test)
show_video(y_pred,My_test,30,model,True)